In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import re
import collections

In [4]:
base_url = 'https://www.imdb.com'
user_ids = ['ur117926588', 'ur15298231', 'ur1994077', 'ur17646017', 'ur4532636', 'ur22171966',
           'ur3223254', 'ur66111139', 'ur63040106', 'ur84924605', 'ur4103165', 
           'ur59627333', 'ur98435364', 'ur65836273','ur44059846', 'ur7813355', 
           'ur98240498', 'ur59184301', 'ur57691865', 'ur3793011']
top_3_ratings = []

In [5]:
for user_id in user_ids:
    url = f'{base_url}/user/{user_id}/?ref_=tt_urv'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    # find all rows in histogram
    histogram = soup.find("div", {"class": "histogram-vertical"})
    rows = histogram.find_all("div", {"class": "row"})

    # find all rows in year histogram
    histograms = soup.find_all("div", {"class": "histogram-vertical"})
    year_histogram = histograms[1].find_all("div", {"class": "row"})

    # initialize genre and year scores
    genre_scores = collections.defaultdict(int)
    year_scores = collections.defaultdict(int)

    # loop through rows and extract genre and year scores
    for row in rows:
        r = row.find_all("div", {"class": "count"})
        score = r[0].text.strip()
        count = r[1].text.strip()
        genre = row.find("div", {"class": "key"}).text.strip()
        genre_scores[genre] += float(score)

    for row in year_histogram:
        r = row.find_all("div", {"class": "count"})
        score = r[0].text.strip()
        count = r[1].text.strip()
        year = row.find("div", {"class": "key"}).text.strip()
        year_scores[year] += float(score)

    # find top 3 genres and years
    top_genres = [genre for genre, score in sorted(genre_scores.items(), key=lambda x: x[1], reverse=True)[:3]]
    top_years = [year for year, score in sorted(year_scores.items(), key=lambda x: x[1], reverse=True)[:3]]

    # create top_3_ratings list
    top_3_ratings.append({"UserID":user_id, "Top 3 Genres": top_genres, "Top 3 Years": top_years})



In [6]:
# create dataframe
df = pd.DataFrame(top_3_ratings)

In [ ]:
# export dataframe
df.to_csv("user_movie_preferences_20users.csv", index = False, encoding='utf-8-sig')